In [5]:
import keras #DeepLearning Library
import numpy as np 
import pandas as pd

In [6]:
from hyperopt import STATUS_OK,tpe,hp,Trials,fmin  ## This library is used mainly in Hyper parameter tuning in Depp Learning projects

In [7]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [10]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")
data.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6
7,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
8,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
9,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6


In [11]:
##Creating the ANN (Artificial Neural Network)

#Split the data
train ,test = train_test_split(data,test_size=0.20)
train.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
834,7.3,0.205,0.31,1.70,0.060,34.0,110.0,0.99630,3.72,0.69,10.5,6
1455,6.9,0.340,0.74,11.20,0.069,44.0,150.0,0.99680,3.00,0.81,9.2,5
11,8.6,0.230,0.40,4.20,0.035,17.0,109.0,0.99470,3.14,0.53,9.7,5
2124,7.7,0.390,0.28,4.90,0.035,36.0,109.0,0.99180,3.19,0.58,12.2,7
1090,8.3,0.210,0.58,17.10,0.049,62.0,213.0,1.00060,3.01,0.51,9.3,6
1061,7.0,0.210,0.28,7.50,0.070,45.0,185.0,0.99660,3.34,0.55,9.4,5
3012,6.8,0.250,0.37,3.10,0.026,29.0,93.0,0.99035,3.14,0.45,12.2,6
1797,6.9,0.250,0.27,9.05,0.039,37.0,128.0,0.99360,3.27,0.34,11.3,8
1227,6.9,0.180,0.38,6.50,0.039,20.0,110.0,0.99430,3.10,0.42,10.5,5
364,6.8,0.160,0.40,2.30,0.037,18.0,102.0,0.99230,3.49,0.42,11.4,7


In [34]:
# Create the Training and Testing data for both X and Y values

train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel() #Here ravel changes the muti-dimension array into single dimension

#Test Data
test_x = test.drop(['quality'],axis=1).values
test_y = test['quality'].values.ravel()

#Splitting the train data into Train and Validation
train_x, valid_x, train_y, valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=20)

signature = infer_signature(train_x,train_y)

train_x.size

34474

In [35]:
#ANN Model

import mlflow.tensorflow


def train_model(params, epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    #Define the model architecture
    mean = np.mean(train_x,axis=0)
    variance = np.var(train_x,axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=variance),
            keras.layers.Dense(64,activation='relu'),      
            keras.layers.Dense(1)
        ]
    )

    #Compilte the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params['lr'],momentum=params['momentum']
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    #Train the ANN model with lr and momentum params with MLFlow Tracking

    with mlflow.start_run(nested=True):
        print("Train X shape:", train_x.shape)
        print("Train Y shape:", train_y.shape)
        print("Valid X shape:", valid_x.shape)
        print("Valid Y shape:", valid_y.shape)
        print("Test X shape:", test_x.shape)
        print("Test Y shape:", test_y.shape)
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),epochs=epochs,batch_size=64)

        #Evaluate the model
        eval_result = model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse = eval_result[1]

        ##Log the parameters
        mlflow.log_params(params)

        ##Log the metrics
        mlflow.log_metric("eval_rmse",eval_rmse)

        ##Log the model
        mlflow.tensorflow.log_model(
            model,"model",
            signature=signature
        )

        return {"loss":eval_rmse,"status":STATUS_OK,"model":model}

In [36]:
def objective(params):
    #MLFlow will trach the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [37]:
params = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [38]:
import mlflow.tensorflow
import mlflow.types


mlflow.set_experiment("/wine-quality")

with mlflow.start_run():
    #Conduct the hyperparameter searching using Hyperopt
    trials = Trials()
    best=fmin(
        fn=objective,
        space=params,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    #Fetch the details of the best run
    best_run = sorted(trials.results,key=lambda x:x['loss'])[0]

    #Log the best parameters, loss and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)

    #Print out the best parameters and corresponding loss
    print(f"Best Parameter: {best}")
    print(f"Best eval rmse: {best_run["loss"]}")

Train X shape:                                       
(3134, 11)                                           
Train Y shape:                                       
(3134,)                                              
Valid X shape:                                       
(784, 11)                                            
Valid Y shape:                                       
(784,)                                               
Test X shape:                                        
(980, 11)                                            
Test Y shape:                                        
(980,)                                               
Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 8s 177ms/step - loss: 41.5578 - root_mean_squared_error: 6.4465
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40.9852 - root_mean_squared_error: 6.4019 - val_loss: 38.8812 - val_root_mean_squared_error: 6.2355

Epoch 2/3                                            

 1/

In [46]:
import mlflow
logged_model = 'runs:/0044c419d8e648c384ee8bbecda8be72/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(test_x)

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step


array([[6.679511 ],
       [5.5157375],
       [5.6533117],
       [5.3653326],
       [5.909273 ],
       [6.2286777],
       [5.370948 ],
       [6.3666534],
       [6.683653 ],
       [6.612633 ],
       [5.4940996],
       [6.383482 ],
       [5.7373447],
       [6.4065437],
       [6.1458483],
       [6.751807 ],
       [6.4315357],
       [5.2984123],
       [6.3473396],
       [5.8576756],
       [5.6487637],
       [5.5226946],
       [6.4023037],
       [7.3136168],
       [5.9419403],
       [6.270258 ],
       [6.274207 ],
       [6.2898064],
       [5.6700897],
       [5.2240133],
       [5.8456573],
       [5.898524 ],
       [5.9610405],
       [6.5744114],
       [6.374689 ],
       [6.5250907],
       [5.931014 ],
       [6.2829304],
       [6.010499 ],
       [6.7491865],
       [5.897606 ],
       [5.9834013],
       [5.6808186],
       [5.3834248],
       [6.111149 ],
       [6.22178  ],
       [6.085188 ],
       [6.2941446],
       [6.203369 ],
       [5.435342 ],
